# Modeling Database Ingestion and Query

## Environment Setup

For this example, I will be creating a local modeling database that will live at `/database/local_store.db`. If you look at the `bash.sh` and `jupyter.sh` scripts for this project you will see how I am mounting those drives so that I can reuse this local database both on my base OS and in the basic docker image for this project.

In [1]:
import pandas as pd
import pathlib
import json

# run these uploads locally
serv = "local"
configs = pathlib.Path("/database/configs.json")
with open(configs) as read_in:
    configs = json.load(read_in)

# setting local store path
configs[serv][serv]["database"] = str(pathlib.Path("/database/local_store.db"))

In [2]:
import os
import json
import shutil
import numpy as np
from orator import DatabaseManager
from modelingdbtools import query
from modelingdbtools import ingest
from modelingdbtools.utils import admin
from modelingdbtools.schemas import modeling

# create database connection
db = DatabaseManager(configs[serv])
# create and fill tables with basic data
modeling.create_schema(db)
modeling.add_schema_data(db)

Created table: SourceType
Created table: User
Created table: Iota
Created table: Dataset
Created table: IotaDatasetJunction
Created table: Run


## Input Dataset

The next cell will create an ingestion test dataset and a csv copy of the dataset saved to `/active/fp_example/example.csv` to show the capabilities of how data can be pushed to each instance of a modeling database using `modelingdbtools`.

In [3]:
# create test upload dataset
fp_ex = pathlib.Path("/active/fp_example/")
if not os.path.exists(fp_ex):
    os.makedirs(fp_ex)

test = []
for i in range(10):
    fp =  fp_ex / (str(i) + ".json")
    with open(fp, "w") as write_out:
        json.dump({"hello": "world"}, write_out)
    
    d = {}
    d["strings"] = "foo" + str(i)
    d["bools"] = np.random.rand() < 0.5
    d["floats"] = np.random.rand() * 100
    d["ndarrays"] = np.random.rand(2)
    d["tuples"] = tuple([1, 2, 3])
    d["sets"] = set([1, 2, 3, 3, 3])
    d["files"] = str(fp)
    test.append(d)

test = pd.DataFrame(test)
test.to_csv(fp_ex / "example.csv")
test

,bools,files,floats,ndarrays,sets,strings,tuples
0,True,/active/fp_example/0.json,43.747389,"[0.5643305755878399, 0.6632140606844696]","{1, 2, 3}",foo0,"(1, 2, 3)"
1,False,/active/fp_example/1.json,81.263884,"[0.7750176716004773, 0.7465286293692389]","{1, 2, 3}",foo1,"(1, 2, 3)"
2,False,/active/fp_example/2.json,14.450579,"[0.07404816355589705, 0.19810288785157115]","{1, 2, 3}",foo2,"(1, 2, 3)"
3,True,/active/fp_example/3.json,2.169538,"[0.4869247194197047, 0.3565979644712097]","{1, 2, 3}",foo3,"(1, 2, 3)"
4,False,/active/fp_example/4.json,61.400925,"[0.8442405148718909, 0.460814158230027]","{1, 2, 3}",foo4,"(1, 2, 3)"
5,False,/active/fp_example/5.json,81.686670,"[0.4561273303621597, 0.5840947991105992]","{1, 2, 3}",foo5,"(1, 2, 3)"
6,True,/active/fp_example/6.json,60.853102,"[0.5909996382449292, 0.19973470221660228]","{1, 2, 3}",foo6,"(1, 2, 3)"
7,True,/active/fp_example/7.json,86.281845,"[0.22382903044628077, 0.580813218405511]","{1, 2, 3}",foo7,"(1, 2, 3)"
8,True,/active/fp_example/8.json,28.590351,"[0.7780179492759283, 0.5806633350387581]","{1, 2, 3}",foo8,"(1, 2, 3)"
9,False,/active/fp_example/9.json,47.813695,"[0.3672095127812093, 0.7167949078477823]","{1, 2, 3}",foo9,"(1, 2, 3)"


### Uploading the Dataset

What we just did was make a dataset with various types of values to show that the database will handle all classes of data that can be stringified. There are various reasons for doing this, mainly execution time to query, merge, and join datasets when we only want certain columns from each row being the largest. This could have been done in NoSQL but would have required much more time to develop and as a starting proof-of-concept this worked well to get it into peoples hands and testing the capabilities. Development on a NoSQL version may come later but for now SQL is suiting us fine.

With various types created however let's upload and see how the database handles the data ingestion.

In [4]:
# upload the test dataset to the connected db
# map each column to a python class/ object
# indicate not to upload the files to the fms
# indicate which files should be checked for existance
ds_info = ingest.upload_dataset(database=db,
                                dataset=test,
                                name="test_dataset",
                                description="this is the hello world of dataset ingestion",
                                type_map={"bools": bool, 
                                          "files": str, 
                                          "floats": float, 
                                          "ndarrays": np.ndarray, 
                                          "strings": str},
                                upload_files=False,
                                filepath_columns=["files"])
ds_info

,DatasetId,Description,Name
0,1,this is the hello world of dataset ingestion,test_dataset


Before any of this information is stored, the entire dataset is validated for two things, does all the data match the typing provided by the `type_map` parameters and do all the files exist in the columns provided by the `filepath_columns` parameter. Once the dataset is validated, what is actually happening on the backend is that the dataframe is being broken into individual key value pairings each with a source, sourcetype, group (row), valuetype, and creation datetime. This allows you to reduce down datasets to only keys (columns) you care about incredibly quickly, and return back to the original data typing when ingestion occured.

If you want to validate a dataframe but do not want to upload the dataset to the database you can use:

```
from modelingdbtools.utils import checks
checks.validate_dataset(dataset, type_map, filepath_columns)
```

I built a casting function that casts the string values stored back to their original types (in my opinion it covers pretty much all the types that you should really stick into the database, anything else and I think you may be trying to shove too much into the database... ([details on value casting here](./testing_decode_types.ipynb))

To get the dataset pass either `ds_info["DatasetId"]` or `ds_info["Name"]` into `modelingdbtools.query.get_dataset()`

In [5]:
# get the dataset we just uploaded
query.get_dataset(db, id=ds_info["DatasetId"][0])

,bools,files,floats,ndarrays,sets,strings,tuples
0,True,/active/fp_example/0.json,43.747389,"[0.56433058, 0.66321406]","{1, 2, 3}",foo0,"(1, 2, 3)"
1,False,/active/fp_example/1.json,81.263884,"[0.77501767, 0.74652863]","{1, 2, 3}",foo1,"(1, 2, 3)"
2,False,/active/fp_example/2.json,14.450579,"[0.07404816, 0.19810289]","{1, 2, 3}",foo2,"(1, 2, 3)"
3,True,/active/fp_example/3.json,2.169538,"[0.48692472, 0.35659796]","{1, 2, 3}",foo3,"(1, 2, 3)"
4,False,/active/fp_example/4.json,61.400925,"[0.84424051, 0.46081416]","{1, 2, 3}",foo4,"(1, 2, 3)"
5,False,/active/fp_example/5.json,81.686670,"[0.45612733, 0.5840948]","{1, 2, 3}",foo5,"(1, 2, 3)"
6,True,/active/fp_example/6.json,60.853102,"[0.59099964, 0.1997347]","{1, 2, 3}",foo6,"(1, 2, 3)"
7,True,/active/fp_example/7.json,86.281845,"[0.22382903, 0.58081322]","{1, 2, 3}",foo7,"(1, 2, 3)"
8,True,/active/fp_example/8.json,28.590351,"[0.77801795, 0.58066334]","{1, 2, 3}",foo8,"(1, 2, 3)"
9,False,/active/fp_example/9.json,47.813695,"[0.36720951, 0.71679491]","{1, 2, 3}",foo9,"(1, 2, 3)"


This same ingestion process can handle a csv filepath as well however there are some limitations in that when `pandasDataFrame`s are sent to a csv they lose complex typing such as `numpy.ndarray`. In this case we will specify that the `ndarrays` column should be validated as `string`.

Validation of `string` can be done by explictly stating so, or by leaving the that key in the `type_map` blank.

In [6]:
# you can also upload this with a csv filepath
# you may lose typing on complex types though (ndarrays -> str)
ds_info = ingest.upload_dataset(database=db,
                                name="test_dataset",
                                dataset=(fp_ex / "example.csv"),
                                description="this is the hello world of dataset ingestion",
                                type_map={"bools": bool, 
                                          "files": str, 
                                          "floats": float, 
                                          "strings": str},
                                upload_files=False,
                                filepath_columns=["files"])
ds_info

A dataset with that name already exists. Adding new version.


,DatasetId,Description,Name
0,2,this is the hello world of dataset ingestion,test_dataset@@2018-06-14 00:26:44.819390


Let's try filtering down the dataset we just uploaded to only return the `ndarray`s but before we do so, note that because we said to use the same dataset name as before, this upload was given a version (datetime) attached to the name. Additionally, if you want to get all the info regarding each `Iota` then use the `get_info_items` parameter.

Note: `columns` parameter can be passed a `string` or `list of strings`

In [7]:
# get ndarrays columns from the dataset we just uploaded
query.get_dataset(db, id=ds_info["DatasetId"][0], columns="ndarrays", get_info_items=True)

,ndarrays,ndarrays(IotaId),ndarrays(SourceId),ndarrays(SourceTypeId),ndarrays(Type)
0,[0.56433058 0.66321406],75,test_dataset@@2018-06-14 00:26:44.819390,1,<class 'str'>
1,[0.77501767 0.74652863],83,test_dataset@@2018-06-14 00:26:44.819390,1,<class 'str'>
2,[0.07404816 0.19810289],91,test_dataset@@2018-06-14 00:26:44.819390,1,<class 'str'>
3,[0.48692472 0.35659796],99,test_dataset@@2018-06-14 00:26:44.819390,1,<class 'str'>
4,[0.84424051 0.46081416],107,test_dataset@@2018-06-14 00:26:44.819390,1,<class 'str'>
5,[0.45612733 0.5840948 ],115,test_dataset@@2018-06-14 00:26:44.819390,1,<class 'str'>
6,[0.59099964 0.1997347 ],123,test_dataset@@2018-06-14 00:26:44.819390,1,<class 'str'>
7,[0.22382903 0.58081322],131,test_dataset@@2018-06-14 00:26:44.819390,1,<class 'str'>
8,[0.77801795 0.58066334],139,test_dataset@@2018-06-14 00:26:44.819390,1,<class 'str'>
9,[0.36720951 0.71679491],147,test_dataset@@2018-06-14 00:26:44.819390,1,<class 'str'>


### Minimum Upload Requirements

Below is an example of the minimum parameters needed to successfully upload a dataset.

In [8]:
# minimum you need to pass to upload a dataset
ds_info = ingest.upload_dataset(database=db, dataset=test)
ds_info

,DatasetId,Description,Name
0,3,None,maxfield@@2018-06-14 00:26:45.493605


Notice that the dataset name genereated was the `{user}@@{upload_datetime}`. If you are doing a minimal upload with a filepath instead of a `pandas.DataFrame` the name is generated using `{absolute_filepath}@@{upload_datetime}`.

In [9]:
# get the dataset we just uploaded
out = query.get_dataset(db, id=ds_info["DatasetId"][0])

In [10]:
out.applymap(lambda x: type(x))

,bools,files,floats,ndarrays,sets,strings,tuples
0,<class 'bool'>,<class 'str'>,<class 'float'>,<class 'numpy.ndarray'>,<class 'set'>,<class 'str'>,<class 'tuple'>
1,<class 'bool'>,<class 'str'>,<class 'float'>,<class 'numpy.ndarray'>,<class 'set'>,<class 'str'>,<class 'tuple'>
2,<class 'bool'>,<class 'str'>,<class 'float'>,<class 'numpy.ndarray'>,<class 'set'>,<class 'str'>,<class 'tuple'>
3,<class 'bool'>,<class 'str'>,<class 'float'>,<class 'numpy.ndarray'>,<class 'set'>,<class 'str'>,<class 'tuple'>
4,<class 'bool'>,<class 'str'>,<class 'float'>,<class 'numpy.ndarray'>,<class 'set'>,<class 'str'>,<class 'tuple'>
5,<class 'bool'>,<class 'str'>,<class 'float'>,<class 'numpy.ndarray'>,<class 'set'>,<class 'str'>,<class 'tuple'>
6,<class 'bool'>,<class 'str'>,<class 'float'>,<class 'numpy.ndarray'>,<class 'set'>,<class 'str'>,<class 'tuple'>
7,<class 'bool'>,<class 'str'>,<class 'float'>,<class 'numpy.ndarray'>,<class 'set'>,<class 'str'>,<class 'tuple'>
8,<class 'bool'>,<class 'str'>,<class 'float'>,<class 'numpy.ndarray'>,<class 'set'>,<class 'str'>,<class 'tuple'>
9,<class 'bool'>,<class 'str'>,<class 'float'>,<class 'numpy.ndarray'>,<class 'set'>,<class 'str'>,<class 'tuple'>


Even though we didn't validate the dataset with a `type_map` the types were stored on ingestion and cast back on query.

### Upload Failure

There are a couple things that could make the upload fail, the primary two however are: 

1) if a value type doesn't match the specified value type in the `type_map` parameter

2) if a filepath from one of the columns specified in the `filepath_columns` parameter does not exist

In [11]:
# upload failure
# a dataset will not be uploaded if any data or filepath validation fails
ds_info = ingest.upload_dataset(database=db,
                                dataset=test,
                                name="test_dataset",
                                description="this is the hello world of dataset ingestion",
                                # let's change the types in type_map
                                type_map={"bools": float, 
                                          "files": int, 
                                          "floats": bool, 
                                          "ndarrays": pd.DataFrame, 
                                          "strings": type(None)},
                                upload_files=False,
                                filepath_columns=["files"])
ds_info

TypeError: 

Allowed types: <class 'float'>
Given type: <class 'bool'>
Given value: True


In [12]:
# let's remove the files and try to upload the dataset
shutil.rmtree(fp_ex)

ds_info = ingest.upload_dataset(database=db,
                                dataset=test,
                                name="test_dataset",
                                description="this is the hello world of dataset ingestion",
                                type_map={"bools": bool, 
                                          "files": str, 
                                          "floats": float, 
                                          "ndarrays": np.ndarray, 
                                          "strings": str},
                                upload_files=False,
                                filepath_columns=["files"])
ds_info

FileNotFoundError: 

The provided filepath does not exist.
Given filepath: /active/fp_example/0.json
